In [15]:
import os
if os.path.basename(os.getcwd()) != 'biometria2':
    os.chdir(os.path.dirname(os.getcwd()))
import torch
import splitfolders
from tqdm import tqdm
import random

from Speaker_Verification.speech_embedder_net import SpeechEmbedder
from notebooks.experiments_utils import make_experent
from notebooks.experiments_utils import SpeakerDataset

In [28]:
THRESHOLD = 0.3
AUTH_USERS = ['Daniil_Hardzetski', 'Bianka_Kowalska', 'Hubert_Baran']
embedder_net = SpeechEmbedder()
embedder_net.load_state_dict(torch.load(os.path.join('.', 'speech_id_checkpoint', 'ckpt_epoch_300_batch_id_30.pth')))

<All keys matched successfully>

### preparing data
___

In [ ]:
splitfolders.ratio('our_data/nasze', output='our_data/split_data', seed=1337, ratio=(.7, .3)) 

In [7]:
train_dataset = SpeakerDataset('our_data/split_data/train/*/*.wav', device='cpu')
val_dataset = SpeakerDataset('our_data/split_data/val/*/*.wav', device='cpu')

### testing against same user
___

In [24]:
for i, user in enumerate(AUTH_USERS):
    print(user)
    decisions = (make_experent(THRESHOLD, embedder_net(train_dataset[i]), embedder_net(val_dataset[i])))
    print(decisions)
    print("TP = {:.2f}%".format(decisions.count(1)/len(decisions) * 100))
    

Daniil_Hardzetski
[1, 1, 1, 1, 1, 1, 0]
TP = 85.71%
Bianka_Kowalska
[1, 1, 1, 1]
TP = 100.00%
Hubert_Baran
[0, 1, 1]
TP = 66.67%


### testing against random users
___

In [33]:
from typing import Iterable 
#from collections import Iterable                            # < py38


def flatten(items):
    """Yield items from any nested iterable; see Reference."""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            for sub_x in flatten(x):
                yield sub_x
        else:
            yield x

In [35]:
def get_random_profile():
    random_dataset = SpeakerDataset('split_data/val/*/*.wav', device='cpu')
    random_id = random.choices(range(len(random_dataset)), k=1)
    return random_dataset[random_id[0]]


for i, user in enumerate(AUTH_USERS):
    print(user)
    decisions = []
    for j in range(3):
        decisions.append(make_experent(THRESHOLD, embedder_net(get_random_profile()), embedder_net(val_dataset[i])))
    decisions = list(flatten(decisions))
    print(decisions)
    print("TP = {:.2f}%".format(decisions.count(1)/len(decisions) * 100))

Daniil_Hardzetski
[0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0]
TP = 38.10%
Bianka_Kowalska
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
TP = 16.67%
Hubert_Baran
[0, 0, 0, 0, 0, 0, 0, 0, 1]
TP = 11.11%


### testing against noised data
___

In [37]:
NOISES = ['barking_city', 'white_noise_1', 'white_noise_3']

In [43]:
for i, user in enumerate(AUTH_USERS):
    print(f"== {user} ==")
    for noise in NOISES:
        print(f'testing against {noise}')
        noised_dataset = SpeakerDataset(f'our_data/{noise}/*/*.wav', device='cpu')
        decisions = make_experent(THRESHOLD, embedder_net(noised_dataset[i]), embedder_net(train_dataset[i]))
        print(decisions)
        print("TP = {:.2f}%".format(decisions.count(1)/len(decisions) * 100))

== Daniil_Hardzetski ==
testing against barking_city
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1]
TP = 86.67%
testing against white_noise_1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
TP = 100.00%
testing against white_noise_3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TP = 0.00%
== Bianka_Kowalska ==
testing against barking_city
[1, 1, 1, 1, 1, 1, 1]
TP = 100.00%
testing against white_noise_1
[1, 1, 1, 1, 0, 1, 1]
TP = 85.71%
testing against white_noise_3
[0, 1, 1, 1, 0, 0, 0]
TP = 42.86%
== Hubert_Baran ==
testing against barking_city
[1, 0, 1, 0, 0, 1, 0]
TP = 42.86%
testing against white_noise_1
[1, 0, 1, 0, 1, 0, 0]
TP = 42.86%
testing against white_noise_3
[0, 0, 0, 0, 1, 0, 0]
TP = 14.29%
